In [1]:
# In order to make this script working is necessary 
# to have installed the python packages listed below.
# In particular for awkde is possible to insall it with:

# git clone https://github.com/mennthor/awkde
# pip install [--user] [-e] ./awkde

# For more details please consult awkde github page at: https://github.com/mennthor/awkde

In [2]:
# Please notice: the dataset attached to this script consist only a subset of the entire dataset. 
# In particular, weekday 23 time slot.

#Also, on this dataset for the seek of brevity are already applied but not reported
# all the preprocessing steps including temporal and spatial one, in order to obtain data
# ready to be fitted by the model

In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
from awkde import GaussianKDE

In [4]:
dataset = pd.read_csv('Amsterdam_dataset_WE23.csv')

In [5]:
# eventhough also WGS84 coordinate system is available, UTM ones are preferred
# because it's the best choice to use default euclidean metric distance for GridSearchCV

In [6]:
kde_columns_utm_2d = ['start_longitude_utm', 'start_latitude_utm']
#start longitude and latitude are used for 2 dimensional KDE
#it's also possible to use end point(final destination)
# kde_columns_utm_2d = ['end_longitude_utm', 'end_latitude_utm']
kde_columns_utm_4d = ['start_longitude_utm', 'start_latitude_utm', 'end_longitude_utm', 'end_latitude_utm']
bandwidths_utm = np.geomspace(0.5, 10000, 100)

In [7]:
#optimal fixed bandwidth reaserarch in two dimensional space
for i in range(2):
    if i==0:
        grid_search_fixedKDE_2D =  GridSearchCV(KernelDensity(kernel='gaussian'),
                                    {'bandwidth': bandwidths_utm}, cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_2d].dropna())
        log_likelihood_results_fixedKDE_2D = pd.DataFrame(grid_search_fixedKDE_2D.cv_results_['params'])
        log_likelihood_results_fixedKDE_2D['mean_likelihood_score'] = grid_search_fixedKDE_2D.cv_results_['mean_test_score']
        best_bw = grid_search_fixedKDE_2D.best_params_['bandwidth']
    else:
        grid_search_fixedKDE_2D =  GridSearchCV(KernelDensity(kernel='gaussian'),
                                    {'bandwidth': np.linspace(best_bw/2, best_bw*1.5, 50)}, cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_2d].dropna())

In [8]:
grid_search_fixedKDE_2D.best_params_['bandwidth']

67.94722855767006

In [9]:
#it's now possible to fit a 2D fixed KDE with the optimal bandwidth
fixedKDE_2D = KernelDensity(kernel='gaussian',bandwidth=grid_search_fixedKDE_2D.best_params_['bandwidth']).fit(
    dataset[kde_columns_utm_2d])

In [10]:
#optimal fixed bandwidth reaserarch in four dimensional space
for i in range(2):
    if i==0:
        grid_search_fixedKDE_4D =  GridSearchCV(KernelDensity(kernel='gaussian'),
                                    {'bandwidth': bandwidths_utm}, cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_4d].dropna())
        log_likelihood_results_fixedKDE_4D = pd.DataFrame(grid_search_fixedKDE_4D.cv_results_['params'])
        log_likelihood_results_fixedKDE_4D['mean_likelihood_score'] = grid_search_fixedKDE_4D.cv_results_['mean_test_score']
        best_bw = grid_search_fixedKDE_4D.best_params_['bandwidth']
    else:
        grid_search_fixedKDE_4D =  GridSearchCV(KernelDensity(kernel='gaussian'),
                                    {'bandwidth': np.linspace(best_bw/2, best_bw*1.5, 50)}, cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_4d].dropna())

In [11]:
#it's now possible to fit a 4D fixed KDE with the optimal bandwidth
fixedKDE_4D = KernelDensity(kernel='gaussian',bandwidth=grid_search_fixedKDE_4D.best_params_['bandwidth']).fit(
    dataset[kde_columns_utm_4d])

In [12]:
######################################################################
#                  VARIABLE BANDWIDTH (ADAPTIVE) KDE                 #
######################################################################

In [13]:
# since the AwKDE package applies a transformation on the input data
# in order to return a sample space with zero mean vector an
# identity covariance matrix. As also the bandwidth has to be scaled
# therefore for the global_bw research we start from the optimal founded with
# scikit-learn library and divide it for the sample's average standard deviation in
#diffent dimensions.

In [14]:
average_std = np.sqrt(np.sum([np.power(dataset['start_longitude_utm'].std(),2),
                              np.power(dataset['start_latitude_utm'].std(),2)])/2)

In [15]:
global_bw_std = grid_search_fixedKDE_2D.best_params_['bandwidth']/average_std
global_bw_ranges = np.linspace(global_bw_std/2, global_bw_std*2, 100)

In [16]:
#it's possible to find the optimal global_bw with alpha=0.5 according 
# to silverman law

In [17]:
grid_search_2D_VKDE_alpha05 = GridSearchCV(GaussianKDE(diag_cov=False, alpha=0.5),
                                                        {'glob_bw': global_bw_ranges},
                                                      cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_2d].dropna())

In [18]:
VKDE_2D_alpha05 = GaussianKDE(alpha=0.5, 
                              glob_bw=grid_search_2D_VKDE_alpha05.best_params_['glob_bw'])
VKDE_2D_alpha05.fit(dataset[kde_columns_utm_2d])

(start_longitude_utm    121310.459773
 start_latitude_utm     486298.299518
 dtype: float64,
 array([[4601772.08638535, -912501.53547977],
        [-912501.53547977, 3560398.75234897]]))

In [19]:
# or instead is possible to perform a cross validation  
# with both glob_bw and alpha hyperparamters

In [20]:
grid_search_2D_VKDE = GridSearchCV(GaussianKDE(diag_cov=False),
                                                        {'glob_bw': global_bw_ranges,
                                                        'alpha':[None, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},
                                                      cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_2d].dropna())

In [21]:
VKDE_2D = GaussianKDE(alpha=grid_search_2D_VKDE.best_params_['alpha'], 
                      glob_bw=grid_search_2D_VKDE.best_params_['glob_bw'])
VKDE_2D.fit(dataset[kde_columns_utm_2d])

(start_longitude_utm    121310.459773
 start_latitude_utm     486298.299518
 dtype: float64,
 array([[4601772.08638535, -912501.53547977],
        [-912501.53547977, 3560398.75234897]]))

In [22]:
#################################################
#the same can be replicated in four dimensions
#################################################

In [23]:
grid_search_4D_VKDE_alpha05 = GridSearchCV(GaussianKDE(diag_cov=False, alpha=0.5),
                                                        {'glob_bw': global_bw_ranges},
                                                      cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_4d].dropna())

/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,


In [24]:
VKDE_4D_alpha05 = GaussianKDE(alpha=0.5, 
                              glob_bw=grid_search_4D_VKDE_alpha05.best_params_['glob_bw'])
VKDE_4D_alpha05.fit(dataset[kde_columns_utm_4d])

(start_longitude_utm    121310.459773
 start_latitude_utm     486298.299518
 end_longitude_utm      121678.332133
 end_latitude_utm       486020.938639
 dtype: float64,
 array([[ 4601772.08638535,  -912501.53547977,  1243546.4445405 ,
          -310127.68386297],
        [ -912501.53547977,  3560398.75234897,   -91418.51580534,
           449124.3267851 ],
        [ 1243546.4445405 ,   -91418.51580534,  9048605.42453936,
         -1425545.88456961],
        [ -310127.68386297,   449124.3267851 , -1425545.88456961,
          5391501.0963258 ]]))

In [25]:
grid_search_4D_VKDE = GridSearchCV(GaussianKDE(diag_cov=False),
                                                        {'glob_bw': global_bw_ranges,
                                                        'alpha':[None, 0.1, 0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]},
                                                      cv=10, n_jobs=-1).fit(dataset[kde_columns_utm_4d].dropna())

/opt/anaconda3/envs/bigdatalab_cpu_202101/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: RuntimeWarning: invalid value encountered in subtract
  array_means[:, np.newaxis]) ** 2,


In [26]:
VKDE_4D = GaussianKDE(alpha=grid_search_4D_VKDE.best_params_['alpha'], 
                      glob_bw=grid_search_4D_VKDE.best_params_['glob_bw'])
VKDE_4D.fit(dataset[kde_columns_utm_4d])

(start_longitude_utm    121310.459773
 start_latitude_utm     486298.299518
 end_longitude_utm      121678.332133
 end_latitude_utm       486020.938639
 dtype: float64,
 array([[ 4601772.08638535,  -912501.53547977,  1243546.4445405 ,
          -310127.68386297],
        [ -912501.53547977,  3560398.75234897,   -91418.51580534,
           449124.3267851 ],
        [ 1243546.4445405 ,   -91418.51580534,  9048605.42453936,
         -1425545.88456961],
        [ -310127.68386297,   449124.3267851 , -1425545.88456961,
          5391501.0963258 ]]))